In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Class Data 

In [ ]:
def fn_backtest(df_signal):
    us_bd = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())
    df_backtest = pd.DataFrame({
        'SSI' : df_signal.SSI.where(df_signal['Signal'] != 0).dropna(),
        'Signal' : df_signal.Signal.where(df_signal['Signal'] != 0).dropna(),
        'entry date' : df_signal.index.where(df_signal['Signal'] != 0).dropna() + us_bd,
        'entry price' : None,
        'exit date' : None,
        'exit price' : None,
        },index=df_signal.index.where(df_data['Signal'] != 0).dropna())
    # df_backtest['entry price']=yf_Data.get_yf_data('SSO',df_backtest['entry date'],df_backtest['entry date']+ pd.offsets.BDay(1),"Open")
    df_backtest['entry price'] = df_backtest['entry date'].apply(lambda date: yf_Data.get_yf_data('QLD', date.strftime('%Y-%m-%d'), (date + us_bd).strftime('%Y-%m-%d'), "Open"))
    df_backtest['exit date'] = df_backtest.index.to_frame()['Dates'].apply(lambda date: get_exit_date(df_signal,date))
    df_backtest['exit price'] = df_backtest['exit date'].apply(lambda date: yf_Data.get_yf_data('QLD', date.strftime('%Y-%m-%d'), (date + us_bd).strftime('%Y-%m-%d'), "Open"))
    df_backtest['Trade return %'] = np.where(
        df_backtest['Signal'] == 1,
        df_backtest[['entry price', 'exit price']].pct_change(axis=1)['exit price'] * 100,
        np.where(
            df_backtest['Signal'] == -1,
            df_backtest[['exit price', 'entry price']].pct_change(axis=1)['entry price'] * 100,
            np.nan
        )
    )
    df_backtest['Cumulative Return']=df_backtest['Trade return %'].cumsum()
    return df_backtest
# df_backtest['entry price'].values.to_float

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Fetch historical data
# data = df_SSI_BBG.copy()
data.rename(columns={'SSI': 'Close'}, inplace=True)
symbol = 'SSI'
# Create features
data['Returns'] = data['Close'].pct_change()
data['SMA'] = data['Close'].rolling(window=20).mean()
data['StdDev'] = data['Close'].rolling(window=20).std()
data = data.dropna()


# Define the target
data['Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)

# Define features and target
features = ['Returns', 'SMA', 'StdDev']
X = data[features]
y = data['Target']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)


# Predict signals
data['Signal'] = model.predict(X)

# Calculate strategy returns
data['Strategy_Returns'] = data['Signal'].shift(1) * data['Close'].pct_change()


# Calculate cumulative returns
data['Cumulative_Market_Returns'] = (1 + data['Close'].pct_change()).cumprod()
data['Cumulative_Strategy_Returns'] = (1 + data['Strategy_Returns']).cumprod()

# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(data['Cumulative_Market_Returns'], label='Market Returns')
plt.plot(data['Cumulative_Strategy_Returns'], label='Strategy Returns')
plt.title(f'{symbol} - ML-Based Mean Reversion with Backtesting')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Sample data fetching step
data = df_SSI_BBG.copy()
# data = pd.DataFrame({'SSI': np.random.randn(1000)})  # Placeholder for actual SSI data

# data.rename(columns={'SSI': 'Close'}, inplace=True)
# symbol = 'SSI'

# Create features
data['Returns'] = data['SSI'].pct_change()
data['SMA'] = data['SSI'].rolling(window=20).mean()
data['StdDev'] = data['SSI'].rolling(window=20).std()

# Calculate turning points
data['TurningPoint'] = (np.sign(data['Returns'].shift(1)) != np.sign(data['Returns'])).astype(int)
data = data.dropna()

# Define features and target
features = ['Returns', 'SMA', 'StdDev']
X = data[features]
y = data['TurningPoint']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict probabilities
data['Predicted_TurningPoint'] = model.predict(X)
data['Confidence'] = model.predict_proba(X)[:, 1]  # Probability of being a turning point

# Evaluate the model
accuracy = accuracy_score(y_test, model.predict(X_test))
print(f'Model accuracy: {accuracy:.2f}')

# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(data['SSI'], label='SSI', color='blue')
plt.scatter(data.index, data['SSI'], c='blue', s=data['Confidence'] * 100, label='Predicted Turning Points', alpha=0.6)
plt.title(f'{symbol} - Predicted Turning Points with Confidence Levels')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from ta import add_all_ta_features

# Sample data fetching step
data = df_SSI_BBG.copy()
# data = pd.DataFrame({'SSI': np.random.randn(1000)})  # Placeholder for actual SSI data

data.rename(columns={'SSI': 'Close'}, inplace=True)
symbol = 'SSI'

# Create features
data['Returns'] = data['Close'].pct_change()
data['SMA'] = data['Close'].rolling(window=20).mean()
data['StdDev'] = data['Close'].rolling(window=20).std()

# Add technical indicators as features
data = add_all_ta_features(data, open="Close", high="Close", low="Close", close="Close", volume="Close", fillna=True)

# Calculate turning points
data['TurningPoint'] = (np.sign(data['Returns'].shift(1)) != np.sign(data['Returns'])).astype(int)
data = data.dropna()

# Define features and target
features = [col for col in data.columns if col not in ['Close', 'TurningPoint']]
X = data[features]
y = data['TurningPoint']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# Predict probabilities
data['Predicted_TurningPoint'] = best_model.predict(X)
data['Confidence'] = best_model.predict_proba(X)[:, 1]  # Probability of being a turning point

# Evaluate the model
accuracy = accuracy_score(y_test, best_model.predict(X_test))
print(f'Model accuracy: {accuracy:.2f}')

# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(data['Close'], label='SSI', color='blue')
plt.scatter(data.index, data['Close'], c='red', s=data['Confidence'] * 100, label='Predicted Turning Points', alpha=0.6, edgecolors='k')
plt.title(f'{symbol} - Predicted Turning Points with Confidence Levels')
plt.legend()
plt.grid(True)
plt.show()
